<a href="https://colab.research.google.com/github/osu-mp/ai-539-nlp-group/blob/main/SparsityCheckCondaColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:32
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()


✨🍰✨ Everything looks OK!


In [5]:
import os
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive')
import sys
from google.colab import drive
sys.path.append(GOOGLE_DRIVE_PATH)
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
%load_ext autoreload
%autoreload 2


In [13]:

conda_path = ''
try:
    conda_path = !which conda
finally:
    print('')

if (len(conda_path) == 0):
    print('installing miniconda')
    !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
    !conda update conda -y -q
    !source /usr/local/etc/profile.d/conda.sh
    !conda init 
    !conda install -n root _license -y -q
else:
    print('found miniconda')

conda_envs = !conda env list
res = [i for i in conda_envs if 'test38' in i]
if (len(res) == 0):
    print('not found test38 env', len(res))
    !conda create -y -q --name test38 python=3.8
else:
    print('found test38 env', len(res))

#!conda env update -n base -f '/content/drive/MyDrive/AI539MUSSLT/environment.yml'



found miniconda
not found test38 env 0
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/test38

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-3.1.0              |       h0b41bf4_0         2.5 MB  conda-forge
    python-3.8.16              |he550d4f_1_cpython        21.8 MB  conda-forge
    setuptools-67.6.0          |     pyhd8ed1ab_0         566 KB  conda-forge
    wheel-0.40.0               |     pyhd8ed1ab_0          54 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        24.9 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-for

In [14]:
%%bash
source activate test38

python
import sys
# maybe only need this the first time we run this notebook
sys.path.append('/usr/local/lib/python3.8/site-packages')

print("Python version")
print(sys.version)

Python version
3.8.16 | packaged by conda-forge | (default, Feb  1 2023, 16:01:55) 
[GCC 11.3.0]


In [26]:
!conda env update -n test38 -f '/content/drive/MyDrive/AI539MUSSLT/environment.yml'

Solving environment: - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 22.11.1
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.1.0


Installing pip dependencies: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [ ]:

#!pip install torch==1.7.1
#!pip install editdistance


#!pip install fairseq==0.10.2
#!pip install transformers

In [16]:
import torch
import torch.nn as nn
import os
from transformers import BartForConditionalGeneration

In [17]:
def get_model_sparsity(model: nn.Module) -> float:
    """
    calculate the sparsity of the given model
        sparsity = #zeros / #elements = 1 - #nonzeros / #elements
    """
    num_nonzeros, num_elements = 0, 0
    for param in model.parameters():
        num_nonzeros += param.count_nonzero()
        num_elements += param.numel()
    return 1 - float(num_nonzeros) / num_elements

def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements



In [18]:
import copy
import torch.nn.utils.prune as prune
import copy
def get_weight_parameters(layer):
    '''
    Get all parameters/modules identified as 'weight'
    '''
    weight_parameters = []
    if len(list(layer.children())) > 0:
        for child in layer.children():
            for param in child.named_parameters():
                if 'weight' == param[0]:
                    weight_parameters.append((child, param[0]))
            weight_parameters.extend(get_weight_parameters(child))
    
    
    return weight_parameters


def prune_weight_parameters(model, prune_amount):
    '''
    Global pruning
    '''
    params_to_prune = get_weight_parameters(model)
  
    prune.global_unstructured(
        params_to_prune, 
        pruning_method=prune.L1Unstructured, 
        amount=prune_amount,
    )

    for module, name in params_to_prune:
        try:
            prune.remove(module, name)
            #print(module)
        except Exception as e:
            print(e)
    return model

def get_pruned_models(model, sparsity):
    model_to_prune = copy.deepcopy(model)
    pruned_model = prune_weight_parameters(model, sparsity)
    return pruned_model

In [27]:
# Load the model in fairseq
from fairseq.models.bart import BARTModel
bart = BARTModel.from_pretrained('drive/MyDrive/AI539MUSSLT/bart.base', checkpoint_file='checkpoint_best.pt')
bart.eval()  # disable dropout (or leave in train mode to finetune)

ModuleNotFoundError: ignored

In [ ]:
for module_name, module in bart.named_modules():
  print(module_name)

In [ ]:
bart.fill_mask(['The cat <mask> on the <mask> .'], topk=3, beam=10)

In [ ]:
get_model_sparsity(bart.model)

In [ ]:
bart_total_params = sum(p.numel() for p in bart.parameters() if p.requires_grad)
bart_total_params

In [ ]:
bart_pruned = get_pruned_models(bart.model, 0.1)
print(get_model_sparsity(bart_pruned))
for i in range(2):
  bart_pruned = get_pruned_models(bart_pruned, 0.1*(i+1))
  print(get_model_sparsity(bart_pruned))

In [ ]:
torch.save(bart_pruned.state_dict(), '/content/test.pt')

In [ ]:
model = BARTModel.from_pretrained('drive/MyDrive/AI539MUSSLT/bart.base', checkpoint_file='checkpoint_best.pt')
model.model.load_state_dict(torch.load('/content/test.pt'), strict=False)

model.eval()
get_model_sparsity(model)


In [ ]:
#/content/drive/MyDrive/AI539MUSSLT/fairseq_models/checkpoint_6_40000.pt
model = BARTModel.from_pretrained('drive/MyDrive/AI539MUSSLT/bart.base', checkpoint_file='checkpoint_best.pt')
model.model.load_state_dict(torch.load('/content/drive/MyDrive/AI539MUSSLT/fairseq_models/checkpoint_6_40000.pt'), strict=False)

model.eval()
get_model_sparsity(model)